# Filtering the Yelp Dataset

This notebook contains steps for filtering the Yelp business dataset into a dataset containing only open restaurants in Idaho. The reviews for those restaurants are then added to the dataset.

## 1. Download the files    

Download the dataset at https://www.yelp.com/dataset.

## 2. Define input and output files.

The Yelp business and review json files are the inputs, and the output is a csv containing the filtered and merged data.

In [2]:
in_filename_businesses = 'yelp_academic_dataset_business.json'
in_filename_reviews = 'yelp_academic_dataset_review.json'
out_filename = 'yelp_filtered_dataset.csv'

## 3. Filter the business dataset to only include open restaurants in Boise, Idaho.

For the remaining steps, I relied heavily on the instructions in https://towardsdatascience.com/converting-yelp-dataset-to-csv-using-pandas-2a4c8f03bd88.         

In [ ]:
import pandas as pd

In [4]:
# Read the business file into a dataframe
df_business = pd.read_json(in_filename_businesses, lines=True)

# Filter the business dataframe to only include open restaurants in Boise
df_restaurants = df_business[df_business['categories'].str.contains('Restaurants', case=False, na=False)]
df_open_restaurants = df_restaurants[df_restaurants['is_open']==1]
df_Boise_restaurants = df_open_restaurants[df_open_restaurants['city']=='Boise']

# Drop unneeded columns
columns_to_drop = ['address', 'city', 'state', 'latitude', 'longitude', 'is_open', 'hours']
df_Boise_restaurants = df_Boise_restaurants.drop(columns_to_drop, axis=1)

## 4. Merge the relevant reviews into the dataset.

In [6]:
# Read the reviews file in chunks of 100,000. Set each column to the appropriate data type. Both of these steps reduce memory usage
df_reviews = pd.read_json(in_filename_reviews, orient='records', lines=True,
                       dtype={'review_id':str,'user_id':str,
                              'business_id':str,'stars':int,
                              'date':str,'text':str,'useful':int,
                              'funny':int,'cool':int},
                       chunksize=100000)

In [7]:
# Read the review data and merge them to the business data in chunks
chunk_list = []
chunk_number = 0
for chunk in df_reviews:
     # Remove unneeded columns
     chunk = chunk.drop(['user_id', 'review_id', 'useful','funny','cool'], axis=1)
     # Rename the 'stars' column since the restaurants dataframe also has a 'stars' column
     chunk = chunk.rename(columns={'stars': 'review_stars'})
     # Inner merge with restaurants dataframe to include only the reviews for those restaurants
     merged_chunk = pd.merge(df_Boise_restaurants, chunk, on='business_id', how='inner')
     # Show feedback on progress
     chunk_number += 1
     print(f"Working on chunk {chunk_number}...")
     chunk_list.append(merged_chunk)

Working on chunk 1...
Working on chunk 2...
Working on chunk 3...
Working on chunk 4...
Working on chunk 5...
Working on chunk 6...
Working on chunk 7...
Working on chunk 8...
Working on chunk 9...
Working on chunk 10...
Working on chunk 11...
Working on chunk 12...
Working on chunk 13...
Working on chunk 14...
Working on chunk 15...
Working on chunk 16...
Working on chunk 17...
Working on chunk 18...
Working on chunk 19...
Working on chunk 20...
Working on chunk 21...
Working on chunk 22...
Working on chunk 23...
Working on chunk 24...
Working on chunk 25...
Working on chunk 26...
Working on chunk 27...
Working on chunk 28...
Working on chunk 29...
Working on chunk 30...
Working on chunk 31...
Working on chunk 32...
Working on chunk 33...
Working on chunk 34...
Working on chunk 35...
Working on chunk 36...
Working on chunk 37...
Working on chunk 38...
Working on chunk 39...
Working on chunk 40...
Working on chunk 41...
Working on chunk 42...
Working on chunk 43...
Working on chunk 44.

In [8]:
# Concatenate the chunks into one dataframe
df_yelp = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

## 5. Export the filtered and merged dataset to a CSV

In [ ]:
df_yelp.to_csv(out_filename, index=False)